### IMPORTS

In [1]:
import numpy as np

In [2]:
from sklearn.linear_model import LinearRegression

In [3]:
import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Dense, LSTM, Dropout

In [4]:
import ipynb.fs.full.splitting as splitting

### LINEAR REGRESSION

In [5]:
def linear_regression(data, settings):
    
    # INSTANTIATE MODEL
    linear = LinearRegression()
    
    # TRAIN WITH TRAIN DATA
    model = linear.fit(data['train']['features'], data['train']['labels'])
    
    # PREDICT USING TEST DATA
    predictions = model.predict(data['test']['features'])
    
    return np.ndarray.flatten(predictions)

### LONG-SHORT TERM MEMORY

In [14]:
def lstm_add(model, data, index, name, settings, shape):
    
    # AVAILABLE LSTM LAYERS
    available = {
        'lstm': LSTM,
        'dropout': Dropout,
        'dense': Dense
    }
    
    # SELECT THE CORRECT FUNCTION
    func = available[name]
    
    # IF AN ACTIVATION IS FOUND & THIS IS THE FIRST LAYER
    if 'activation' in settings and index == 0:
        model.add(func(
            settings['value'],
            activation=settings['activation'],
            input_shape=shape
        ))
    
    # JUST AN ACTIVATION FUNCTION
    elif 'activation' in settings:
        model.add(func(
            settings['value'],
            activation=settings['activation']
        ))
        
    # OTHERWISE, DEFAULT TO JUST USING THE VALUE
    else:
        model.add(func(
            settings['value']
        ))

In [13]:
def long_short_term(data, settings):
    
    # INSTANTIATE MODEL
    model = Sequential()
    
    generator = splitting.morph(
        data['train']['features'],
        data['train']['labels'],
        settings['morph']
    )
    
    # LOOP THROUGH REQUESTED MODEL LAYERS
    for index, layer in enumerate(settings['layers']):
        
        # LAYER PROPS
        name = list(layer)[0]
        params = layer[name]
        
        # GENERATE & ADD THE LAYER
        lstm_add(model, data, index, name, params, generator[0][0].shape)
    
    # COMPILE THE MODEL
    model.compile(
        loss=settings['loss'],
        optimizer=settings['optimizer']
    )
    
    model.fit_generator(generator, steps_per_epoch=len(generator), epochs=settings['epochs'], verbose=0)
    
    # TRAIN USING TRAIN DATA
    #model.fit(
    #    features,
    #    labels,
    #    epochs=settings['epochs'],
    #    batch_size=settings['batch'],
    #)
    
    foob = splitting.morph(
        data['test']['features'],
        data['test']['labels'],
        settings['morph']
    )
    
    # PREDICT USING TEST DATA
    predictions = model.predict(data['test']['features'])
    
    return np.ndarray.flatten(predictions)

### START TRAINING A MODEL

In [8]:
def start(dataset, name, settings):
    
    # AVAILABLE MODELS
    available = {
        'linreg': linear_regression,
        'lstm': long_short_term
    }
    
    # SELECT THE CORRECT FUNCTION & START
    return available[name](dataset, settings)